In [1]:
import os

import torch
import pandas as pd
from ipywidgets import Button, Text, HBox, VBox, HTML, Dropdown, IntSlider
from IPython.display import display, HTML as IPyHTML, clear_output

from retrieval.models import load_colbert_and_tokenizer, ColBERTInference, TfIdf
from retrieval.indexing import IndexConfig, index, ColBERTRetriever
from retrieval.data import Passages
from retrieval.model_understanding import visualize

In [2]:
def getWiklinkToPIDs(wikis, passages, pids):
    print(passages, wikis, pids)
    return [wikis.loc[passages[passages['PID'] == pid].WID.values[0]].url for pid in pids]

def getPassagesForPIDs(passages, pids):
    return [passages[passages['PID'] == pid].passage.values[0] for pid in pids]

def display_text_with_link(counter, html_address, text):
    html_link = f'<div style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; border-radius: 5px; margin-bottom: 10px;"><div style="display: inline-block; width: 30px; text-align: center; margin-right: 10px;">{counter}.</div><a href="{html_address}" target="_blank" style="color: #1a0dab; text-decoration: none;">{text}</a></div>'
    display(HTML(html_link))

def display_best_search_results(question, pairs):
    title = f'<h2 style="color: #1a0dab;"><i class="fa fa-search" style="margin-right: 10px;"></i>Best search results for question: "{question}"</h2>'
    display(HTML(title))

    for i, (website_html, text) in enumerate(pairs, start=1):
        display_text_with_link(i, website_html, text)

In [3]:
%%html
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">

In [5]:
# let the user select some the database and model
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

CHECKPOINT_PATHS = ["data/colbertv2.0"]
WIKI_PATHS = ["data/fandoms_qa/harry_potter/all", "data/fandoms_qa/witcher/all", "data/fandoms_qa/fandoms_all/all"]

dropdown_checkpoint = Dropdown(options=CHECKPOINT_PATHS, description='Select a model checkpoint:')
display(dropdown_checkpoint)

dropdown_wiki = Dropdown(options=WIKI_PATHS, description='Select a wiki:')
display(dropdown_wiki)

Dropdown(description='Select a model checkpoint:', options=('data/colbertv2.0',), value='data/colbertv2.0')

Dropdown(description='Select a wiki:', options=('data/fandoms_qa/harry_potter/all', 'data/fandoms_qa/witcher/a…

In [6]:
# initialize the models and index the selected wiki
passages_path = os.path.join(dropdown_wiki.value, "passages.tsv")
wiki_json_path = os.path.join(dropdown_wiki.value, "wiki.json")
checkpoint_path = dropdown_checkpoint.value
print(f"Passages: {passages_path}")


index_cfg = IndexConfig(
        passages_path=passages_path,
        checkpoint_path=checkpoint_path,
        use_gpu="cpu" not in DEVICE,
        device=DEVICE,
        dtype=torch.float16,
        batch_size=8
)
print(index_cfg)
passages = Passages(passages_path)

inference = ColBERTInference.from_pretrained(checkpoint_path)
inference.tokenizer.doc_maxlen = 512
# we can't skip the punctuation, otherwise the visualization would be offset
inference.colbert.config.skip_punctuation = False
inference.colbert.skiplist = None
retriever = ColBERTRetriever(inference, device=DEVICE, passages=passages)
retriever.indexer = index(inference, index_cfg, store=False)

[2023-06-23 15:00:36][WARNING] colbert_config.json does not exist, loading default config.
[2023-06-23 15:00:36][INFO] Detected BERT Tokenizer. Using unused tokens for [Q]/[D] tokens
[2023-06-23 15:00:36][INFO] Detected BERT Tokenizer. Using unused tokens for [Q]/[D] tokens
[2023-06-23 15:00:36][WARNING] colbert_config.json does not exist, loading default config.


Passages: data/fandoms_qa/witcher/all/passages.tsv
IndexConfig(passages_path='data/fandoms_qa/witcher/all/passages.tsv', checkpoint_path='data/colbertv2.0', index_path='', batch_size=8, use_gpu=True, device='cuda', dtype=torch.float16)


[2023-06-23 15:00:38][INFO] Detected ColBERTv2 checkpoint. Loading the model!
[2023-06-23 15:00:41][INFO] Successfully loaded weights for last ColBERTv2 layer!
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3130/3130 [00:41<00:00, 75.56it/s]


In [7]:
passages_df = pd.read_csv(passages_path, sep ='\t')
wikis_df = pd.read_json(wiki_json_path, orient='index')

In [8]:
def search(question, k, mode):
    if mode.lower() == "tf_idf":
        sims, pids  = retriever.tf_idf_rank([question], k=k)[0]
    elif mode.lower() == "rerank":
        sims, pids  = retriever.rerank([question], k=k)[0]
    elif mode.lower() == "full_retrieval":
        sims, pids  = retriever.full_retrieval([question], k=k)[0]
    else:
        raise ValueError()
    
    links = getWiklinkToPIDs(wikis_df, passages_df, pids.tolist())
    passage_matches = getPassagesForPIDs(passages_df, pids.tolist())

    return list(zip(links, sims.tolist(), passage_matches))

In [10]:
clear_output()

def display_text_with_link(counter, html_address, sim, text):
    html_link = f'<div style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; border-radius: 5px; margin-bottom: 10px;"><div style="display: inline-block; width: 30px; text-align: center; margin-right: 10px;">{counter}.</div><div>{round(sim, 4)}</div><a href="{html_address}" target="_blank" style="color: #1a0dab; text-decoration: none;">{text}</a></div>'
    display(HTML(html_link))

def update_pairs(button):
    question = text_input.value

    # Call your Python function 'f' to update the pairs list based on the question
    updated_pairs = search(question, k=topK_slider.value, mode=dropdown_mode.value)

    # Clear the existing output, including search results
    clear_output()

    # Display the input field, magnifying glass button, "Best search results for question" block
    display(VBox([input_box]))

    # Display the updated search results
    for i, (website_html, sim, text) in enumerate(updated_pairs, start=1):
        display_text_with_link(i, website_html, sim, text)

# Create a slider where the user can select the maximal amount of passages that should be returned"  
topK_slider = IntSlider(min=10, max=1000, step=10, description='Select how many passages should be returned:')
display(topK_slider)

# Create a slider where the user can select the desired quering methode
dropdown_mode = Dropdown(options=["tf_idf", "rerank", "full_retrieval"], description='Select a quering methode:')
display(dropdown_mode)
        
# Create the input field and magnifying glass button
text_input = Text(description='', layout={'width': '500px', 'height': '75px'}, style={'font-size': '18px'})
search_button = Button(description='', icon='search', layout={'width': '40px', 'height': '40px'})

# Add the click event handler to the magnifying glass button
search_button.on_click(update_pairs)

# Create a horizontal box to hold the input field and magnifying glass button
input_box = HBox([text_input, search_button])

# Create the "Best search results for question" block
best_results_title = HTML('<h2 style="color: #1a0dab;">Best search results for question:</h2>')

# Display the input field, magnifying glass button, "Best search results for question" block
display(VBox([input_box]))



# example queries:
#   How many children did Harry Potter have?
#   Where was Rowling born?
#   Where was the author of Harry Potter born?


HTML(value='<div style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; border-radius: 5px; …

HTML(value='<div style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; border-radius: 5px; …

HTML(value='<div style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; border-radius: 5px; …

HTML(value='<div style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; border-radius: 5px; …

HTML(value='<div style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; border-radius: 5px; …

HTML(value='<div style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; border-radius: 5px; …

HTML(value='<div style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; border-radius: 5px; …

HTML(value='<div style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; border-radius: 5px; …

HTML(value='<div style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; border-radius: 5px; …

HTML(value='<div style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; border-radius: 5px; …

In [11]:
clear_output()

test_query = "How many children did Harry Potter have?"
test_passage = '"[Main family] James Potter was the only child of Fleamont and Euphemia Potter. James married Lily Evans and they had one child, Harry, who famously defeated Lord Voldemort. Harry eventually married Ginny Weasley and they had three children: James Sirius, Albus Severus, and Lily Luna Potter."'
html_vis = visualize(inference, test_query, test_passage, k=2, similarity="cosine")
display(HTML(html_vis[0]))
display(HTML(html_vis[1]))
display(HTML(html_vis[2]))

HTML(value='<span style="background-color:#f7fbffff"> [CLS]</span><span style="background-color:#f7fbffff"> [D…

HTML(value='<span style="background-color:#4896c8ff"> [CLS]</span><span style="background-color:#f7fbffff"> [D…

HTML(value='<span style="background-color:#4896c8ff"> [CLS]</span><span style="background-color:#f7fbffff"> [D…

In [12]:
clear_output()

test_query = "How many children did Harry Potter have?"
test_passage = '["December 29 2007, 12:00am, The Times] "She felt compelled to map out the futures of the surviving members of the Weasley/Potter clan in a family tree. Readers will know that Harry and Ginny marry and have three children. Luna Lovegood, Harry’s dreamy friend, produces two children, with her naturalist husband, Rolf. Bill Weasley and Fleur Delacour have three children, and George Weasley sires a daughter, and a son named Fred after his twin who died in the battle of Hogwarts. “I can’t help it,” the author says in JK Rowling . . . a Year in the Life. “It was like running a race and you get to the finishing line and you’re running too fast to stop, so I do know what happened afterwards and I couldn’t stop my imagination doing that."'
html_vis = visualize(inference, test_query, test_passage, k=2, similarity="cosine")
display(HTML(html_vis[0]))
display(HTML(html_vis[1]))
display(HTML(html_vis[2]))

HTML(value='<span style="background-color:#f7fbffff"> [CLS]</span><span style="background-color:#f7fbffff"> [D…

HTML(value='<span style="background-color:#4896c8ff"> [CLS]</span><span style="background-color:#f7fbffff"> [D…

HTML(value='<span style="background-color:#4896c8ff"> [CLS]</span><span style="background-color:#f7fbffff"> [D…